# Tissue Patterning

## 1. Emergent properties of collective gene-expression patterns in multicellular systems
Smart, M., & Zilman, A. (2023). Emergent properties of collective gene-expression patterns in multicellular systems. *Cell Reports Physical Science*, 101247.
[Paper Link](https://doi.org/10.1016/j.xcrp.2023.101247)
### 1.1 Single-cell model
Assume cell state is described by the activity of $N$ genes in the cell. Let $\xi_1, \ldots, \xi_N$ denotes the desired cell types, with $\xi_i\in\{1,-1\}^N$. Let $\hat{J}=\xi(\xi^{T}\xi)^{-1}\xi^T$ represent the gene-gene interactions. 
* First, we want to show that $\xi_i$ is the eigenvector of matrix $\hat{J}$. That is to show that $$\hat{J}\xi_i=\lambda_i\xi_i.$$
Let $A=(\xi^{T}\xi)^{-1}\xi^T$. Note that 
$$A\xi=(\xi^{T}\xi)^{-1}\xi^T\xi=I.$$
This implies the following,
\begin{align*}
\hat{J}\xi&=\xi A \xi\\
&=\xi I\\
&=I\xi.
\end{align*}
Therefore, $\xi_i$ are eigenvectors of matrix $\hat{J}$ for the repeated eigenvaule $\lambda=1$.


* Next, to get the final gene-gene interaction matrix J, the authors set the diagonal value to zero. That is,
$$J=\hat J - \text{diag}(\hat J).$$
Assuming all entries of matrix $\hat J$ are the same, the above transformation won't change the eigenspace of the matrix. But, is it guaranteed all diagonal entries are the same? 

In [1]:
import numpy as np
import random
from numpy.linalg import inv
from scipy.stats import bernoulli
from numpy import linalg as LA

# Construct the gene-gene interaction matrix J
xi1 = np.array([1,1,-1,1])
xi2 = np.array([-1,1,1,1])

# xi1 = np.array([ 1, 1, 1,-1,-1,-1,-1,-1,-1])
# xi2 = np.array([-1,-1,-1, 1,-1,-1,-1,-1,-1])
# xi3 = np.array([-1,-1,-1,-1,-1,-1, 1, 1, 1])

xi = np.stack((xi1, xi2), axis=-1)
Jhat = xi@inv(xi.T@xi)@xi.T
J = Jhat - np.diag(np.diag(Jhat))
print(Jhat@xi2)

[-1.  1.  1.  1.]


### 1.2 Single Cell Hamiltonian

$$\mathcal{H}(s_0)= -\frac12(s^TJs) - h^Ts$$

In [2]:
# Generate all possible states of the cell and verify the lowest energy state in a single cell
xn = xi1.shape[0]
comb = np.tile([-1,1], (xn, 1)).tolist()
S = np.stack(np.meshgrid(*comb), -1).reshape(-1, xn).T
H = np.diag(-S.T@J@S/2)
print("The lowest energy states in a single cell are the coloumn vectors of matrix\n", S[:,np.where(H == H.min())])

The lowest energy states in a single cell are the coloumn vectors of matrix
 [[[-1  1 -1  1]]

 [[-1 -1  1  1]]

 [[ 1 -1  1 -1]]

 [[-1 -1  1  1]]]


### 1.3 Multicellular Model

Assume a given tissue (multicellular stystem) has $M$ different cells. Let $s_i\in\{1,-1\}^N$ represent the $i$-th cell. The Hamiltonian of the system is calculated as the following,
$$\mathcal{H}(s_1,s_2,\ldots, s_M)=\sum_i^M -\frac12(s_i^TJs_i)+\gamma \sum_i\sum_j A_{ij}f(s_i,s_j).$$
* The first term describes the summation of the Hamiltonian of each cell
* The second term describes the Hamiltonian from cell-cell interaction,
$$f(s_i,s_j)=-\frac12 s_i^TWs_j,$$
with strength $\gamma$ and randomly sampled matrix $W$.


In [3]:
# Cell arrangement: n columns of m cells
n = 5
m = 5
N = n*m

# Generating neighborhood matrix A
A = np.zeros((N,N))
k = 0
for i in np.arange(1,n+1):
    for j in np.arange(1,m+1):
        B = np.zeros((n+2,m+2))
        B[i-1:i+2,j-1:j+2] = 1
        B[i,j] = 0
        B_temp = B[1:n+1,1:m+1]
        A[k,:] = B_temp.flatten()
        k +=1

# Ramdon inter-cellular communication matrix W
np.random.seed(100)
W = np.random.uniform(-1,1,(4,4))

In [4]:
T = 100                          # Simulation Time Steps
s_all = np.zeros((xn,N,T+1))     # Storing Transitions
gamma=10                         # Cell-cell communication strength
beta = 200                       # Noise strength

# Initial tissue state
for i in range(N):
    r = np.random.randint(len(S.T))
    s_all[:,i,0] = S.T[r]
    #s_all[:,i,0] = [1,1,-1,1]

# Stochastic update one gene at a time
for k in range(1,T+1):
    for i in range(N):
        s = np.copy(s_all[:,i,k-1])
        cell_trans = s
        for j in range(4):
            ind = A[i,:] != 0
            h = J@s.T + gamma*np.sum(s@W@s_all[:,ind,k-1])
            transition_prob = 1/(1 + np.exp(-2*beta*h[j]))
            cell_trans[j] = 2*bernoulli.rvs(transition_prob)-1
            s_all[j,i,k] = cell_trans[j]


/var/folders/js/cp98kts10mq5xh9jhj49174h0000gp/T/ipykernel_98402/4177037945.py:20: RuntimeWarning: overflow encountered in exp
  transition_prob = 1/(1 + np.exp(-2*beta*h[j]))


In [5]:
# conda install ipywidgets for movie 
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
%matplotlib inline

## Definition of the plot_tissue function, our "callback function".
def plot_tissue(t):
    cell_color = [1,2,4,8]@np.heaviside(s_all[:,:,t],0)
    cell_color.resize((n, m))
    plt.imshow(cell_color,vmin=0,vmax=15)
    plt.colorbar(ticks=np.arange(0,2**4,4))
    plt.show()

## Generate our user interface.
interact(plot_tissue, t=IntSlider(min=0, max=T, step=1, value=0));

interactive(children=(IntSlider(value=0, description='t'), Output()), _dom_classes=('widget-interact',))